# 01. Seq2Seq 기반 기계 번역기 (Machine Translation)

## 🤖 Seq2Seq(Sequence to Sequence)란?
**Seq2Seq**는 입력된 시퀀스(문장)를 끝까지 듣고 문맥을 파악한 뒤, 다른 도메인의 시퀀스(번역문)로 변환해주는 딥러닝 모델 아키텍처입니다. 기계 번역, 챗봇, 텍스트 요약 등에 아주 널리 쓰이는 기초적인 모델이에요.

### 🧠 핵심 원리: 인코더(Encoder)와 디코더(Decoder)
Seq2Seq는 크게 두 부분으로 나뉩니다. 통역사에 비유해서 설명해볼게요.

1.  **인코더 (Encoder - 듣는 사람)**: 
    - 외국어(영어) 문장을 끝까지 주의 깊게 듣습니다. 
    - 문장의 모든 단어를 순서대로 처리한 뒤, 문장 전체의 의미를 압축한 **'단 하나의 생각(Context Vector)'**을 만들어냅니다. 
    - 이 Context Vector에는 문장의 핵심 정보가 숫자로 요약되어 있죠.

2.  **디코더 (Decoder - 말하는 사람)**: 
    - 인코더가 건네준 **Context Vector(압축된 생각)**를 첫 단서로 받습니다.
    - 이를 바탕으로 모국어(한국어) 문장을 한 단어씩 생성해냅니다.
    - "아, 앞 단어가 '나는'이었으니까 다음은 '학교에'가 자연스럽겠군"하며 문맥을 이어갑니다.

자, 이제 이 멋진 통역사를 텐서플로우(TensorFlow)로 직접 구현해봅시다!

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models

# 랜덤 시드 고정 (재현성을 위해)
np.random.seed(42)
tf.random.set_seed(42)

## 1. 사전 학습된 임베딩 다운로드 (GloVe)
단어를 숫자로 바꿀 때, 미리 학습된 **GloVe** 임베딩을 사용하면 모델이 단어 사이의 의미 관계(예: 왕-남자 = 여왕-여자)를 더 잘 이해할 수 있습니다. 100차원 벡터를 사용할 거예요.

In [2]:
!gdown 1qk-14tgVHPXT5jfRUE4Ua2ji4EXwS022

## 2. 데이터 준비 (영어 - 한국어)
번역기 학습을 위해 영어 문장과 그에 대응하는 한국어 번역문 쌍이 필요합니다.
여기서는 `fra.txt` (원래 프랑스어 예제지만 여기선 한국어 데이터로 가정하고 처리) 같은 텍스트 파일이나 CSV를 읽어옵니다. 편의상 데이터를 직접 정의하거나 로드하는 부분입니다.

In [3]:
# 예제 데이터셋 로드 (실제 파일 경로에 맞게 수정 필요)
# 여기서는 학습 흐름을 보여주기 위해 소량의 샘플 데이터를 사용한다고 가정하겠습니다.
# 실제로는 수만 개의 문장 쌍이 필요합니다.

# 데이터 예시 (영어, 한국어)
raw_data = [
    ("I go.", "나는 간다."),
    ("He runs.", "그는 달린다."),
    ("She eats.", "그녀는 먹는다."),
    ("I am a student.", "나는 학생이다."),
    ("You represent.", "너는 대표한다."),
    ("Do you like singing?", "노래하는 거 좋아해요?"),
    ("Tom could've been seriously injured.", "톰은 심각한 부상을 당할 수도 있었다."),
    ("This is my school.", "이곳은 내 학교다."),
    ("Social media is a waste of time.", "소셜 미디어는 시간 낭비야."),
    ("I'm really sorry.", "정말 미안해."),
    ("My hair is black.", "내 머리카락은 검은색이야."),
    ("French is interesting.", "프랑스어는 재미있어."),
    ("I like bus.", "나는 버스를 좋아해."),
    ("Let's study", "공부하자."),
    ("I ate sandwich", "나는 샌드위치를 먹었어.")
]

# 데이터 분리
eng_mn = [x[0] for x in raw_data]
kor_mn = [x[1] for x in raw_data]

# 데이터 확인
print(f"영어 문장 개수: {len(eng_mn)}")
print(f"한국어 문장 개수: {len(kor_mn)}")
print(eng_mn[:3])
print(kor_mn[:3])

### 2-1. 특수 토큰 추가 (<sos>, <eos>)
디코더(번역을 생성하는 부분)는 어디서 말을 시작하고, 어디서 끝맺어야 할지 알아야 합니다.
그래서 한국어 문장(Target)에는 앞뒤로 **특수 신호**를 붙여줍니다.

*   **`<sos>` (Start Of Sequence)**: "자, 이제 통역 시작한다!" (시작 신호)
*   **`<eos>` (End Of Sequence)**: "여기까지가 끝이야." (종료 신호)

**훈련할 때 데이터 형태:**
*   **디코더 입력 (Input)**: `<sos>` 나는 학교에 간다 -> (현재 상태를 보고 다음 단어 예측용)
*   **디코더 정답 (Target)**: 나는 학교에 간다 `<eos>` -> (모델이 맞춰야 할 정답)

In [4]:
# 디코더 입력: <sos>로 시작
kor_inputs = ["<sos> " + sent for sent in kor_mn]
# 디코더 정답(타겟): <eos>로 끝남
kor_targets = [sent + " <eos>" for sent in kor_mn]

eng_inputs = eng_mn  # 인코더 입력은 그대로 사용

print("Decoder Input:", kor_inputs[:3])
print("Decoder Target:", kor_targets[:3])

## 3. 정수 인코딩 (Integer Encoding)
컴퓨터는 'Apple', '사과' 같은 글자를 이해하지 못합니다. 그래서 **단어 집합(Vocabulary)**을 만들고, 각 단어마다 고유한 **번호(ID)**를 부여하는 과정이 필요합니다.

In [5]:
# 하이퍼파라미터 설정
VOCAB_SIZE = 10000  # 사용할 최대 단어 수

# --- 영어 토크나이저 (Encoder Input) ---
eng_tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token='<OOV>')
eng_tokenizer.fit_on_texts(eng_inputs)          # 영문 데이터로 단어장 생성
eng_inputs_seq = eng_tokenizer.texts_to_sequences(eng_inputs)  # 텍스트 -> 숫자 시퀀스 변환

# --- 한국어 토크나이저 (Decoder Input/Target) ---
# filters='' : <sos>, <eos> 같은 특수 문자가 걸러지지 않도록 필터링 제거
kor_tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token='<OOV>', filters='') 
kor_tokenizer.fit_on_texts(kor_inputs + kor_targets)   # 한국어 전체 데이터로 단어장 생성

kor_inputs_seq = kor_tokenizer.texts_to_sequences(kor_inputs)
kor_targets_seq = kor_tokenizer.texts_to_sequences(kor_targets)

print("영어 단어장 크기:", len(eng_tokenizer.word_index))
print("한국어 단어장 크기:", len(kor_tokenizer.word_index))

# 실제 사용할 단어 수 확인
eng_num_words = min(VOCAB_SIZE, len(eng_tokenizer.word_index))
kor_num_words = min(VOCAB_SIZE, len(kor_tokenizer.word_index))

## 4. 패딩 (Padding) - 길이 맞추기
신경망은 입력 길이를 똑같이 맞춰주는 것을 좋아합니다. 문장 길이가 다르면 짧은 문장의 빈 공간을 **0**으로 채워줍니다.

### ⚠️ 중요: 패딩 위치 (Pre vs Post)
*   **Encoder Input (영어)**: `padding='pre'` (앞에 0 채움)
    - LSTM 같은 순환신경망은 마지막 입력의 영향을 가장 크게 받습니다. 번역에서 보통 문장의 끝부분(동사 등)이 중요하거나, 입력의 끝과 출력의 시작 거리를 좁히기 위해 인코더는 앞에 패딩을 넣는 경우가 많습니다.
*   **Decoder Input (한국어)**: `padding='post'` (뒤에 0 채움)
    - 디코더는 `<sos>`부터 시작해서 순서대로 문장을 생성해야 하므로, 문장이 끝나고 뒤쪽에 빈 공간이 있어야 자연스럽습니다.

In [6]:
# 최대 문장 길이 설정 (여기선 데이터에 맞춰 자동 계산)
eng_max_len = max([len(seq) for seq in eng_inputs_seq])
kor_max_len = max([len(seq) for seq in kor_inputs_seq])

print(f"영어 최대 길이: {eng_max_len}, 한국어 최대 길이: {kor_max_len}")

# 패딩 적용
eng_inputs_padded = pad_sequences(eng_inputs_seq, maxlen=eng_max_len, padding='pre')   # 인코더는 앞쪽 패딩
kor_inputs_padded = pad_sequences(kor_inputs_seq, maxlen=kor_max_len, padding='post')  # 디코더는 뒤쪽 패딩
kor_targets_padded = pad_sequences(kor_targets_seq, maxlen=kor_max_len, padding='post')

## 5. GloVe 임베딩 매트릭스 생성
사전 학습된 GloVe 벡터를 가져와서, 우리 단어장에 있는 단어들에 매칭시켜줍니다. 이렇게 하면 모델이 처음부터 맨땅에 헤딩하지 않고, 단어의 의미를 어느 정도 알고 시작할 수 있습니다.

In [7]:
EMBEDDING_DIM = 100

# GloVe 파일 읽어서 딕셔너리로 저장
embeddings_index = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print(f'{len(embeddings_index)}개의 GloVe 단어 벡터 로드 완료.')

# 우리 영어 단어장에 맞춰 임베딩 매트릭스 생성
en_embedding_matrix = np.zeros((eng_num_words + 1, EMBEDDING_DIM))
for word, i in eng_tokenizer.word_index.items():
    if i > eng_num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        en_embedding_matrix[i] = embedding_vector

## 6. Seq2Seq 모델 구축 (Training Model)

### 6-1. 인코더 (Encoder)
인코더의 역할은 문장을 끝까지 읽고 **'Context Vector'**를 만들어내는 것입니다.
LSTM 레이어에서 **`return_state=True`**가 핵심입니다! 
원래 LSTM은 매 시점의 출력(Output)을 내보내지만, 인코더는 출력값은 필요 없고 **마지막 내부 상태(Hidden State, Cell State)**가 중요합니다. 이 두 상태 값이 바로 문장의 엑기스인 Context Vector입니다.

In [8]:
LATENT_DIM = 256  # 은닉 상태 크기 (Context Vector 차원)

# --- Encoder ---
encoder_inputs = layers.Input(shape=(eng_max_len,), name='Encoder_Input')

# GloVe 임베딩 적용 (trainable=True로 두면 미세 조정 가능)
encoder_embedding = layers.Embedding(eng_num_words + 1, 
                                     EMBEDDING_DIM, 
                                     weights=[en_embedding_matrix], 
                                     trainable=True,
                                     mask_zero=True, # 0 패딩 무시
                                     name='Encoder_Embedding')(encoder_inputs)

# LSTM (return_state=True 필수!)
# encoder_outputs: 매 시점의 출력 (여기선 안 씀)
# state_h: 마지막 Hidden State (단기 기억)
# state_c: 마지막 Cell State (장기 기억)
encoder_outputs, state_h, state_c = layers.LSTM(LATENT_DIM, 
                                                return_state=True, 
                                                name='Encoder_LSTM')(encoder_embedding)

# 인코더의 최종 상태 (Context Vector)
encoder_states = [state_h, state_c]

### 6-2. 디코더 (Decoder)
디코더는 **'Teacher Forcing(교사 강요)'** 방식으로 학습합니다.
정답(`<sos>` 나는 학교에 간다)을 입력으로 주면서, 다음 단어(나는 학교에 간다 `<eos>`)를 맞추도록 훈련시킵니다.

1.  **`initial_state`**: 디코더의 첫 시작 상태를 **인코더의 마지막 상태(Context Vector)**로 설정합니다. 즉, 인코더가 읽은 내용을 바탕으로 시작하는 거죠.
2.  **`return_sequences=True`**: 디코더는 번역문을 한 단어씩 모두 생성해야 하므로, 모든 시점의 출력값을 반환해야 합니다.
3.  **`return_state=True`**: 학습 땐 필요 없지만, 나중에 추론(Inference) 단계에서 다음 단어 예측을 위해 현재 상태를 넘겨받아야 하므로 미리 설정해둡니다.

In [9]:
# --- Decoder ---
decoder_inputs = layers.Input(shape=(kor_max_len,), name='Decoder_Input')

# 디코더 임베딩 (처음부터 학습)
decoder_embedding_layer = layers.Embedding(kor_num_words + 1, 
                                           EMBEDDING_DIM, 
                                           mask_zero=True,
                                           name='Decoder_Embedding')
decoder_embedding = decoder_embedding_layer(decoder_inputs)

# 디코더 LSTM
# initial_state=encoder_states : 인코더의 기억을 이어받음
# return_sequences=True : 모든 시점의 단어를 예측해야 함
decoder_lstm = layers.LSTM(LATENT_DIM, 
                           return_sequences=True, 
                           return_state=True, 
                           name='Decoder_LSTM')

decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# 출력층 (다중 분류)
decoder_dense = layers.Dense(kor_num_words + 1, activation='softmax', name='Decoder_Output')
decoder_outputs = decoder_dense(decoder_outputs)

# --- 전체 모델 정의 ---
# 입력: [영어(인코더), 한국어(디코더)] -> 출력: [한국어(정답)]
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs, name='Seq2Seq_Training')

model.summary()

## 7. 모델 학습 (Training)
여기서는 `sparse_categorical_crossentropy`를 사용해 원-핫 인코딩 없이 정수 인덱스로 바로 오차를 계산합니다.

In [10]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 학습 시작
history = model.fit([eng_inputs_padded, kor_inputs_padded], 
                    kor_targets_padded,
                    batch_size=32,
                    epochs=50,
                    validation_split=0.2)

In [11]:
# 학습 결과 시각화
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy')
plt.legend()
plt.show()

## 8. 추론 모델 (Inference Model) 구성
학습 모델과 추론 모델은 다릅니다! 
학습 땐 정답을 한꺼번에 넣어줬지만(Teacher Forcing), 실제 번역 때는 **이전 시점에 모델이 뱉은 단어를 다시 다음 시점의 입력으로 넣어주는 루프(Loop)**를 돌아야 합니다.

따라서 **인코더**와 **디코더**를 분리해서 각각 별도 모델로 만들어 줍니다.

### 8-1. 인코더 추론 모델
영어 문장을 넣으면 **Context Vector(상태들)**만 뱉어내는 모델입니다. 이 상태값들을 디코더의 초기 연료로 쓸 겁니다.

In [12]:
# 입력: 영어 문장
# 출력: 인코더 상태 (h, c)
encoder_model = models.Model(encoder_inputs, encoder_states)

### 8-2. 디코더 추론 모델
이 모델은 매 스텝마다 하나씩 실행됩니다.
1.  **입력**: 현재 단어 토큰 + 이전 시점의 상태(h, c)
2.  **동작**: 입력받은 상태를 초기 상태로 세팅하고 LSTM을 돌림
3.  **출력**: 다음 단어 예측 확률 + 현재 시점의 새로운 상태(h, c)

이 새로운 상태(h, c)는 다음 단어를 예측할 때 다시 입력으로 들어갑니다. (순환 구조)

In [13]:
# 1. 디코더가 받을 이전 시점의 상태들 정의 (Input Layer)
decoder_state_input_h = layers.Input(shape=(LATENT_DIM,))
decoder_state_input_c = layers.Input(shape=(LATENT_DIM,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 2. 학습된 임베딩 레이어 가져오기 (재사용)
# 추론 땐 한 단어씩 들어오므로 shape는 (1, 1)
decoder_inputs_single = layers.Input(shape=(1,))
decoder_embedding_inference = decoder_embedding_layer(decoder_inputs_single)

# 3. 학습된 LSTM 레이어 가져오기 (재사용)
decoder_outputs_inference, state_h_inf, state_c_inf = decoder_lstm(
    decoder_embedding_inference, initial_state=decoder_states_inputs
)
decoder_states_inference = [state_h_inf, state_c_inf]

# 4. 학습된 Dense 레이어 가져오기 (재사용)
decoder_outputs_inference = decoder_dense(decoder_outputs_inference)

# 5. 모델 생성
# 입력: [현재 단어, 지난 h, 지난 c]
# 출력: [다음 단어 확률, 현재 h, 현재 c]
decoder_inference_model = models.Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs_inference] + decoder_states_inference
)

## 9. 실제 번역기 구동 (Translate Function)
이제 만들어둔 인코더와 디코더를 연결해서 번역을 수행하는 함수입니다.
1.  **인코더**가 영어 문장을 Context Vector로 압축.
2.  **디코더**에 첫 입력으로 `<sos>`와 Context Vector를 주입.
3.  디코더가 내뱉은 단어를 결과에 추가하고, 이번에 내뱉은 상태를 저장.
4.  내뱉은 단어와 저장한 상태를 다시 디코더 입력으로 사용 (무한 루프).
5.  `<eos>`가 나오거나 최대 길이에 도달하면 멈춤.

In [14]:
def decode_sequence(input_seq):
    # 1. 인코더로 입력 문장을 상태 벡터(Context Vector)로 변환
    states_value = encoder_model.predict(input_seq)

    # 2. 디코더의 첫 입력으로 <sos> 토큰 준비
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = kor_tokenizer.word_index['<sos>']

    stop_condition = False
    decoded_sentence = ""

    while not stop_condition:
        # 3. 디코더 모델에 [현재 단어, 이전 상태]를 넣어 예측 수행
        # output_tokens: 다음 단어 확률 분포, h/c: 현재 상태 업데이트
        output_tokens, h, c = decoder_inference_model.predict([target_seq] + states_value)

        # 4. 가장 확률 높은 단어 선택 (Greedy Search)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = kor_tokenizer.index_word.get(sampled_token_index, "<OOV>")

        # 5. 종료 조건 검사 (<eos>에 도달하거나 최대 길이 초과 시)
        if (sampled_word == '<eos>' or len(decoded_sentence) > kor_max_len):
            stop_condition = True
        else:
            decoded_sentence += sampled_word + " "

        # 6. 다음 루프를 위해 입력 업데이트
        # 이번에 예측한 단어가 다음 스텝의 입력이 됨
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 7. 상태 업데이트
        states_value = [h, c]

    return decoded_sentence.strip()

## 10. 테스트: 번역 결과 확인

In [15]:
# 학습 데이터 중 몇 개를 골라 테스트
for seq_index in [0, 1, 2, 5, 8]:
    input_seq = eng_inputs_padded[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    
    print("-" * 35)
    print('입력 문장 (영어):', eng_inputs[seq_index])
    print('정답 문장 (한국어):', kor_mn[seq_index])
    print('번역 결과 (기계):', decoded_sentence)